In [1]:
from src.ga_api_package.ga_api_client import System, Repository, HyperLogLog, pprint
import pandas as pd
# connect to API server
repo = Repository('https://rdlab-214.genie-analytics.com/api', 'api@default', 'api123!@#')
# create adhoc
pipeline = {
    'scope': {
        'i-field': '@controller.home',
        'i-entry': [
            [True,'t:0:0:1',0]
        ],
        'mode': '%directional'
    },
    'metric': [
        ['$sum', {
            'field':'@flow.bytes'
        }]
    ]
}
adhoc = await repo.set_adhoc(300, pipeline)

In [2]:
# query recent 24 hour data
end = pd.Timestamp.now('+08:00').floor('5T')
start = end - pd.Timedelta('24H')
df = await adhoc.query_data(start, '24H', 'hour')
print(df)

                            0         1          2
0   2022-03-14 19:55:00+08:00  28756869   66857137
1   2022-03-14 20:00:00+08:00  28149256   40341406
2   2022-03-14 20:05:00+08:00  26441237   77390001
3   2022-03-14 20:10:00+08:00  65411059  134590623
4   2022-03-14 20:15:00+08:00  34255394  245273141
..                        ...       ...        ...
283 2022-03-15 19:30:00+08:00  29972000  276233789
284 2022-03-15 19:35:00+08:00  22323271  773709726
285 2022-03-15 19:40:00+08:00  18927316  181475285
286 2022-03-15 19:45:00+08:00  25739990   46402767
287 2022-03-15 19:50:00+08:00  17601331   97514552

[288 rows x 3 columns]


In [36]:
df.columns = ['timestamp','forward','opposite']
df2 = df.set_index('timestamp')
threshold = df2.quantile(0.95)
print(threshold)

forward     1.220527e+09
opposite    2.682560e+09
Name: 0.95, dtype: float64


In [37]:
df3 = (df2.assign(f_ratio=df2['forward']/threshold['forward'],
                  o_ratio=df2['opposite']/threshold['opposite']))
print(df3)

                            forward   opposite   f_ratio   o_ratio
timestamp                                                         
2022-03-14 19:55:00+08:00  28756869   66857137  0.023561  0.024923
2022-03-14 20:00:00+08:00  28149256   40341406  0.023063  0.015038
2022-03-14 20:05:00+08:00  26441237   77390001  0.021664  0.028849
2022-03-14 20:10:00+08:00  65411059  134590623  0.053592  0.050172
2022-03-14 20:15:00+08:00  34255394  245273141  0.028066  0.091432
...                             ...        ...       ...       ...
2022-03-15 19:30:00+08:00  29972000  276233789  0.024557  0.102974
2022-03-15 19:35:00+08:00  22323271  773709726  0.018290  0.288422
2022-03-15 19:40:00+08:00  18927316  181475285  0.015507  0.067650
2022-03-15 19:45:00+08:00  25739990   46402767  0.021089  0.017298
2022-03-15 19:50:00+08:00  17601331   97514552  0.014421  0.036351

[288 rows x 4 columns]


In [40]:
print(df3.describe())

            forward      opposite     f_ratio     o_ratio
count  2.880000e+02  2.880000e+02  288.000000  288.000000
mean   2.237944e+08  6.525512e+08    0.183359    0.243257
std    8.656657e+08  9.062278e+08    0.709256    0.337822
min    1.145766e+07  2.934597e+07    0.009387    0.010940
25%    2.549672e+07  6.810091e+07    0.020890    0.025387
50%    3.289263e+07  2.756288e+08    0.026950    0.102748
75%    6.049853e+07  9.018558e+08    0.049568    0.336192
max    9.835610e+09  5.089592e+09    8.058492    1.897289


In [43]:
forward_filter = df3['f_ratio'] > 2
opposite_filter = df3['o_ratio'] > 2
filter = forward_filter | opposite_filter
df4 = df3[filter]
print(df4)

                              forward    opposite   f_ratio   o_ratio
timestamp                                                            
2022-03-15 15:20:00+08:00  5611942115   763194606  4.597965  0.284502
2022-03-15 16:15:00+08:00  4263097711   388243404  3.492833  0.144729
2022-03-15 16:20:00+08:00  9835609724   708055744  8.058492  0.263948
2022-03-15 17:10:00+08:00  3897586423  1149994976  3.193363  0.428693
2022-03-15 17:25:00+08:00  4089303217   775994466  3.350440  0.289274
2022-03-15 17:30:00+08:00  2683886607   528544784  2.198957  0.197030


In [ ]:
if 